<a href="https://colab.research.google.com/github/s-choung/OC_Fine_Tuning_SA/blob/main/sa_data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## OCP installation

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
# Set the path to your pseudopotential files in Google Drive
pp_drive_path = '/content/drive/My Drive/potcars'  # Update this path to your actual directory
# Set the VASP_PP_PATH environment variable
os.environ['VASP_PP_PATH'] = pp_drive_path


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

if not os.path.exists('ocp'):
     !git clone https://github.com/Open-Catalyst-Project/ocp.git
os.chdir('ocp')
!pip install -e .
!pip -q install  lmdb pymatgen==2023.5.10 ase==3.22.1 orjson e3nn==0.2.6 wandb
!pip install submitit

2.1.0+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Cloning into 'ocp'...
remote: Enumerating objects: 11623, done.
remote: Counting objects: 100% (2721/2721), done.
remote: Compressing objects: 100% (603/603), done.
remote: Total 11623 (delta 2398), reused 2161 (delta 2118), pack-reused 8902
Receiving objects: 100% (11623/11623), 173.89 MiB | 41.70 MiB/s, done.
Resolving deltas: 100% (8024/8024), done.
Obtaining file:///content/ocp/Open-Catalyst-Dataset/Open-Catalyst-Dataset/ocp
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for ocp-models (pyproject.toml) ... done
  Created wheel for ocp-models: filename=ocp_models-0.0.3-0.editable-py3-none-any.whl size=3498 sha256=4235c0a6a0dddb5f1d6d87d07927ad861fb35c77ccfa108a

In [ ]:
torch.cuda.is_available()

False

In [ ]:
import json,zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm

from ase.io import read

from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist

from ocpmodels.datasets import SinglePointLmdbDataset
from ocpmodels.preprocessing import AtomsToGraphs
from ocpmodels.models import CGCNN,SchNet
from ocpmodels.datasets import data_list_collater
from ocpmodels.common.relaxation.ase_utils import OCPCalculator


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")


In [ ]:
from ase import Atoms
from ase.io import Trajectory, write
from ase.build import bulk, surface, molecule, add_adsorbate, fcc111
from ase.constraints import ExpCellFilter, StrainFilter, FixAtoms, FixedPlane, FixBondLength
from ase.optimize import LBFGS, BFGS, FIRE
from ase.neb import NEB
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.visualize import view
from ase.build.rotate import minimize_rotation_and_translation
from ase.md import MDLogger
from ase.io.vasp import read_vasp

import pandas as pd
import ipywidgets as widgets
from IPython.display import display_png, Image as ImageWidget
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
import shutil

import glob
from pathlib import Path
from PIL import Image, ImageDraw

from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.io.vasp import Poscar
from pymatgen.core.surface import SlabGenerator
from pymatgen.io.ase import AseAtomsAdaptor

import numpy as np
from scipy.spatial.distance import cdist
def visual(structure, index, max_size=(100, 100), stretch_y=1.0):
    renderer = write('./temp.pov', structure, rotation='15z,-90x')
    renderer.render()
    image_path = './temp.png'
    img = Image.open(image_path)

    # Calculate new size with stretch factor
    new_size = (max_size[0], int(max_size[1] * stretch_y))
    img = img.resize(new_size, Image.LANCZOS)
    display(img)
    # Move files to output directory
    files = ['./temp.ini', './temp.pov', './temp.png']
    destination = './output/'

    # Ensure destination directory exists
    os.makedirs(destination, exist_ok=True)

    for file in files:
        # Remove the file in the destination directory if it exists
        if os.path.isfile(os.path.join(destination, os.path.basename(file))):
            os.remove(os.path.join(destination, os.path.basename(file)))

        shutil.move(file, destination)


In [ ]:
!sudo apt-get install povray  ## visulization program

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
povray is already the newest version (1:3.7.0.10-1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


## load some useful utillities
 code taken from https://open-catalyst-project.github.io/tutorial/fine-tuning/fine-tuning-oxides.html

In [ ]:
os.chdir('/content/ocp')
!wget -q https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt
!wget https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/is2re/schnet_all.pt
checkpoint_path = "/content/ocp/gemnet_t_direct_h512_all.pt"
config_yml_path = "configs/s2ef/all/gemnet/gemnet-dT.yml"



--2023-10-29 06:04:03--  https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/is2re/schnet_all.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.39.6, 13.249.39.25, 13.249.39.82, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.39.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14643781 (14M) [binary/octet-stream]
Saving to: ‘schnet_all.pt.1’

schnet_all.pt.1     100%[===================>]  13.96M  --.-KB/s    in 0.1s    

2023-10-29 06:04:04 (101 MB/s) - ‘schnet_all.pt.1’ saved [14643781/14643781]



In [ ]:
# @title
import ocpmodels as om
from pathlib import Path

def ocp_root():
    """Return the root directory of the installed ocp package."""
    return Path(om.__file__).parent.parent

def ocp_main():
    """Return the path to ocp main.py"""
    return ocp_root() / "main.py"

In [ ]:
# @title
import subprocess
import sys
import numba
import numpy as np
import ase
import e3nn
import pymatgen.core as pc
import torch
import torch.cuda as tc
import torch_geometric as tg
import platform
import psutil

def describe_ocp():
    """Print some system information that could be useful in debugging."""
    print(sys.executable, sys.version)
    print(f'ocp is installed at {ocp_root()}')

    commit_hash = (
            subprocess.check_output(
                    [
                        "git",
                        "-C",
                        om.__path__[0],
                        "describe",
                        "--always",
                    ]
                )
                .strip()
                .decode("ascii")
            )
    print(f'ocp repo is at git commit: {commit_hash}')
    print(f'numba: {numba.__version__}')
    print(f'numpy: {np.version.version}')
    print(f'ase: {ase.__version__}')
    print(f'e3nn: {e3nn.__version__}')
    print(f'pymatgen: {pc.__version__}')
    print(f'torch: {torch.version.__version__}')
    print(f'torch.version.cuda: {torch.version.cuda}')
    print(f'torch.cuda: is_available: {tc.is_available()}')
    if tc.is_available():
        print('  __CUDNN VERSION:', torch.backends.cudnn.version())
        print('  __Number CUDA Devices:', torch.cuda.device_count())
        print('  __CUDA Device Name:',torch.cuda.get_device_name(0))
        print('  __CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
    print(f'torch geometric: {tg.__version__}')
    print()
    print(f'Platform: {platform.platform()}')
    print(f'  Processor: {platform.processor()}')
    print(f'  Virtual memory: {psutil.virtual_memory()}')
    print(f'  Swap memory: {psutil.swap_memory()}')
    print(f'  Disk usage: {psutil.disk_usage("/")}')

In [ ]:
# @title
import urllib
import os
from pathlib import Path
import requests

checkpoints = {
    # Open Catalyst 2020 (OC20)
    'CGCNN 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_200k.pt',
    'CGCNN 2M'	    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_2M.pt',
    'CGCNN 20M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_20M.pt',
    'CGCNN All'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/cgcnn_all.pt',
    'DimeNet 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/dimenet_200k.pt',
    'DimeNet 2M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/dimenet_2M.pt',
    'SchNet 200k'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_200k.pt',
    'SchNet 2M'	    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_2M.pt',
    'SchNet 20M'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_20M.pt',
    'SchNet All'	:'https://dl.fbaipublicfiles.com/opencatalystproject/models/2020_11/s2ef/schnet_all_large.pt',
    'DimeNet++ 200k'   :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_200k.pt',
    'DimeNet++ 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_2M.pt',
    'DimeNet++ 20M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_20M.pt',
    'DimeNet++ All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_02/s2ef/dimenetpp_all.pt',
    'SpinConv 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_12/s2ef/spinconv_force_centric_2M.pt',
    'SpinConv All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/spinconv_force_centric_all.pt',
    'GemNet-dT 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_12/s2ef/gemnet_t_direct_h512_2M.pt',
    'GemNet-dT All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2021_08/s2ef/gemnet_t_direct_h512_all.pt',
    'PaiNN All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_05/s2ef/painn_h512_s2ef_all.pt',
    'GemNet-OC 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_base_s2ef_2M.pt',
    'GemNet-OC All'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_base_s2ef_all.pt',
    'GemNet-OC All+MD'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/gemnet_oc_base_s2ef_all_md.pt',
    'GemNet-OC-Large All+MD' :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_07/s2ef/gemnet_oc_large_s2ef_all_md.pt',
    'SCN 2M'   :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_t1_b1_s2ef_2M.pt',
    'SCN-t4-b2 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_t4_b2_s2ef_2M.pt',
    'SCN All+MD' :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/scn_all_md_s2ef.pt',
    'eSCN-L4-M2-Lay12 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l4_m2_lay12_2M_s2ef.pt',
    'eSCN-L6-M2-Lay12 2M'    :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m2_lay12_2M_s2ef.pt',
    'eSCN-L6-M2-Lay12 All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m2_lay12_all_md_s2ef.pt',
    'eSCN-L6-M3-Lay20 All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_03/s2ef/escn_l6_m3_lay20_all_md_s2ef.pt',
    'EquiformerV2 (83M) 2M'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_83M_2M.pt',
    'EquiformerV2 (31M) All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_31M_ec4_allmd.pt',
    'EquiformerV2 (153M) All+MD'     :'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_06/oc20/s2ef/eq2_153M_ec4_allmd.pt',
    # Open Catalyst 2022 (OC22)
    'GemNet-dT OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gndt_oc22_all_s2ef.pt',
    'GemNet-OC OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_all_s2ef.pt',
    'GemNet-OC OC20+OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt',
    'GemNet-OC trained with `enforce_max_neighbors_strictly=False` #467 OC20+OC22' : 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_05/oc22/s2ef/gnoc_oc22_oc20_all_s2ef.pt',
    'GemNet-OC OC20->OC22'	: 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2022_09/oc22/s2ef/gnoc_finetune_all_s2ef.pt',
    'equiformer' : 'https://dl.fbaipublicfiles.com/opencatalystproject/models/2023_10/oc22/s2ef/eq2_121M_e4_f100_oc22_s2ef.pt'
                }

def list_checkpoints():
    """List checkpoints that are available to download."""
    print('See https://github.com/Open-Catalyst-Project/ocp/blob/main/MODELS.md for more details.')
    for key in checkpoints:
        print(key)
    print('Copy one of these keys to get_checkpoint(key) to download it.')


def get_checkpoint(key):
    """Download a checkpoint.

    key: string in checkpoints.

    Returns name of checkpoint that was saved.
    """
    url = checkpoints.get(key, None)
    if url is None:
        raise Exception('No url found for {key}')

    pt = Path(urllib.parse.urlparse(url).path).name

    if not os.path.exists(pt):
        with open(pt, 'wb') as f:
            print(f'Downloading {url}')
            f.write(requests.get(url).content)
    return pt

In [ ]:
# @title
from pathlib import Path
import numpy as np
from ase.db import connect

def train_test_val_split(ase_db, ttv=(0.8, 0.1, .1), files=('train.db', 'test.db', 'val.db'), seed=42):
    """Split an ase db into train, test and validation dbs.

    ase_db: path to an ase db containing all the data.
    ttv: a tuple containing the fraction of train, test and val data. This will be normalized.
    files: a tuple of filenames to write the splits into. An exception is raised if these exist.
           You should delete them first.
    seed: an integer for the random number generator seed

    Returns the absolute path to files.
    """

    for db in files:
        if os.path.exists(db):
            raise Exception('{db} exists. Please delete it before proceeding.')

    src = connect(ase_db)
    N = src.count()

    ttv = np.array(ttv)
    ttv /= ttv.sum()

    train_end = int(N * ttv[0])
    test_end = train_end + int(N * ttv[1])

    train = connect(files[0])
    test = connect(files[1])
    val = connect(files[2])

    ids = np.arange(1, N + 1)
    rng = np.random.default_rng(seed=42)
    rng.shuffle(ids)

    for _id in ids[0:train_end]:
        row = src.get(id=int(_id))
        train.write(row.toatoms())

    for _id in ids[train_end:test_end]:
        row = src.get(id=int(_id))
        test.write(row.toatoms())

    for _id in ids[test_end:]:
        row = src.get(id=int(_id))
        val.write(row.toatoms())

    return [Path(f).absolute() for f in files]

In [ ]:
# @title
from yaml import load, dump
from yaml import CLoader as Loader, CDumper as Dumper
import torch
import os
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
from io import StringIO
import sys
import contextlib

def generate_yml_config(checkpoint_path, yml='run.yml', delete=(), update=()):
    """Generate a yml config file from an existing checkpoint file.

    checkpoint_path: string to path of an existing checkpoint
    yml: name of file to write to.
    pop: list of keys to remove from the config
    update: dictionary of key:values to update

    Use a dot notation in update.

    Returns an absolute path to the generated yml file.
    """

    # You can't just read in the checkpoint with torch. The calculator does some things to it.
    # Rather than recreate that here I just reuse the calculator machinery. I don't want to
    # see the output though, so I capture it.

    with contextlib.redirect_stdout(StringIO()) as _:
        config = OCPCalculator(checkpoint_path=checkpoint_path).config # there seems an parameter checkpoint changed in OCPcalculator (s.choung)

    for key in delete:
        if key in config and len(key.split('.')) == 1:
            del config[key]
        else:
            keys = key.split('.')
            if keys[0] in config:
                d = config[keys[0]]
            else:
                continue
            if isinstance(d, dict):
                for k in keys[1:]:
                    if isinstance(d[k], dict):
                        d = d[k]
                    else:
                        if k in d:
                            del d[k]

    def nested_set(dic, keys, value):
        for key in keys[:-1]:
            dic = dic.setdefault(key, {})
        dic[keys[-1]] = value

    for _key in update:
        keys = _key.split('.')
        nested_set(config, keys, update[_key])


    out = dump(config)
    with open(yml, 'wb') as f:
        f.write(out.encode('utf-8'))

    return Path(yml).absolute()

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
if not os.path.exists('tutorial'):
     !git clone https://github.com/Open-Catalyst-Project/tutorial.git
!git clone https://github.com/Open-Catalyst-Project/Open-Catalyst-Dataset.git

!pip install -e ./Open-Catalyst-Dataset
os.chdir('Open-Catalyst-Dataset')
!git clone -b OC22_dataset https://github.com/Open-Catalyst-Project/Open-Catalyst-Dataset.git
os.chdir('Open-Catalyst-Dataset')
!pwd

fatal: destination path 'Open-Catalyst-Dataset' already exists and is not an empty directory.
Obtaining file:///content/ocp/Open-Catalyst-Dataset
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ocdata
    Found existing installation: ocdata 0.2.0
    Uninstalling ocdata-0.2.0:
      Successfully uninstalled ocdata-0.2.0
  Running setup.py develop for ocdata
fatal: destination path 'Open-Catalyst-Dataset' already exists and is not an empty directory.
/content/ocp/Open-Catalyst-Dataset/Open-Catalyst-Dataset


### OC22 datageneration scheme으로 내 데이터셋 생성이 가능할까

일담 보류 CPU로 전환.
what im trying to do is generating SA datasets for SAC.
#### 전략
unary oxide들만 뽑아서 random하게 surface 만들고, adsorbate으로 metal 붙이기.


In [ ]:
import json
import unittest, sys, json, itertools, os
# sys.path.append(os.getcwd().replace(os.getcwd().split('/')[-1], ''))
from pymatgen.entries.computed_entries import ComputedStructureEntry
from pymatgen.core.periodic_table import Element
from pymatgen.analysis.phase_diagram import PhaseDiagram
from pymatgen.analysis.structure_matcher import StructureMatcher
from ocdata.oc22_dataset.adsorbate_configs import adslist, OOH_list


json_file_path = "ocdata/oc22_dataset/bulk_oxides_20220621.json"
with open(json_file_path, 'r') as f:
    data = json.load(f)
print(len(data))
unary_list=[]
for dat in data:
  if len(dat['composition']) == 2:
    print(dat['composition'])
    #print(dat['entry_id'])
    unary_list.append(dat)
print(len(unary_list))

elements_to_do = [el.symbol for el in Element if el.is_metal and el.number < 43 and 21 < el.number and el.group > 3 and el.group < 15]
elements_to_do.extend(['Ce', 'Si', 'Al'])
exclude_elements = ['Pb', 'Tl', 'Hg','Cd','Mg']  # Add the symbols of the elements you want to exclude
elements_to_do = [el for el in elements_to_do if el not in exclude_elements]
all_metal_name_list = [el.symbol for el in Element if el.is_metal and el.number < 80 and 25 < el.number and el.group > 6 and el.group < 12]
exclude_elements_2 = ['Tc']  # Add the symbols of the elements you want to exclude
all_metal_name_list = [el for el in all_metal_name_list if el not in exclude_elements_2]

print(all_metal_name_list) # metal i considered for anchoring
print(elements_to_do) # metal i considered for support

screened_unary_list=[]
for dat in unary_list:
    composition = dat['composition']
    element1, element2 = composition.keys()
    # Check if both elements are metals with atomic number less than 84 and not in the 'f' block
    if (element1 in elements_to_do) and (element2 == 'O'):
        screened_unary_list.append(dat)

print(f"Number of unary : {len(unary_list)}")
print(f"Number of unary after screening: {len(screened_unary_list)}")

screened_unary_list[0]
for dat in screened_unary_list:
    print(dat['composition'])

output_file_path = "./unary_list.json"  # The path where you want to save the output JSON
with open(output_file_path, 'w') as output_file:
    json.dump(screened_unary_list, output_file)

Urary 저장 완료

이제 slab과 SAC을 담지시켜보자. random generation

##  generating VASP input

I change the VASP setting like following from the code written by Kevin Tran https://github.com/Open-Catalyst-Project/Open-Catalyst-Dataset/blob/OC22_dataset/ocdata/vasp.py

in the same inputs as codes written in OC22 and LWAVE deleted

```
ldau_luj = {
    "V": {"L": 2, "U": 3.25, "J": 0},
    "Cr": {"L": 2, "U": 3.7, "J": 0},
    "Mn": {"L": 2, "U": 3.9, "J": 0},
    "Fe": {"L": 2, "U": 5.3, "J": 0},
    "Co": {"L": 2, "U": 3.32, "J": 0},
    "Ni": {"L": 2, "U": 6.2, "J": 0},
    "Cu": {"L": 2, "U": 4.0, "J": 0},
    "Mo": {"L": 2, "U": 4.38, "J": 0},
    "W":  {"L": 2, "U": 6.2, "J": 0}
}
# NOTE: this is the setting for slab and adslab
VASP_FLAGS = {'ibrion': 2,
            "gga": "PE",
            "encut": 500,
            "ediff": 1e-4,
            "ediffg": -0.05,
            "isym": 0,
            "symprec": 1e-10,
            "ispin": 2,
            "isif": 0,
            "nsw": 300,
            "lwave": False,
            "ldau_luj": ldau_luj
        }
```



In [ ]:
from ocdata import vasp
import importlib
importlib.reload(vasp)


<module 'ocdata.vasp' from '/content/ocp/Open-Catalyst-Dataset/Open-Catalyst-Dataset/ocdata/vasp.py'>

In [ ]:
from ocdata.oc22_dataset.surface import Slab, SlabGenerator, generate_all_slabs
from pymatgen.entries.computed_entries import ComputedStructureEntry
import time
import random

json_file_path = "unary_list.json"
all_pbx_stable_entries = [ComputedStructureEntry.from_dict(d) for d in \
                          json.load(open(json_file_path))]

def generate_test_bulk(all_pbx_stable_entries):
    entry = random.choice(all_pbx_stable_entries)  # Use random.choice to select a random entry
    test_bulk = entry.structure
    print('Bulk:',test_bulk.formula)  # You can remove this line if you don't want to print the result
    return test_bulk

def generate_random_sa_metal(all_metal_name_list):
    entry = random.choice(all_metal_name_list)  # Use random.choice to select a random entry
    sa = Molecule([entry], [[0, 0, 0]])
    print('SA:',sa.formula)  # You can remove this line if you don't want to print the result
    return sa

In [ ]:
o_ref = Molecule(['O'], [[0, 0, 0]])
print(o_ref.species[0].atomic_radius)

0.6 ang


In [ ]:
from ocdata.oc22_dataset.MXide_adsorption import MXideAdsorbateGenerator
from ocdata.oc22_dataset.termination_generator import get_random_clean_slabs
from pymatgen.core.structure import Structure, Lattice, Molecule
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from ocdata import vasp
import os
from pymatgen.core.periodic_table import ElementBase

out_directory = './output_500'
out_directory_slab = './output_500/slab'
out_directory_adslab = './output_500/adslab'
os.makedirs(out_directory, exist_ok=True)
os.makedirs(out_directory_adslab, exist_ok=True)
os.makedirs(out_directory_slab, exist_ok=True)

o_ref = Molecule(['O'], [[0, 0, 0]])
o_ref_radii = o_ref.species[0].atomic_radius
slab_list=[]
adslab_list =[]
def get_tol(bulk,SA):

  sa_radius = SA.species[0].atomic_radius
  elements = bulk.composition.elements
  metal = [element for element in elements if element.name != 'O']
  bulk_metal_radius = metal[0].atomic_radius
  min_m_m_dist=sa_radius + bulk_metal_radius
  min_m_o_dist=o_ref_radii+sa_radius
  tol=min_m_m_dist/min_m_o_dist
  print(tol)
  return tol

count = 0
total_suf_sa_num = 300 # total_suf_sa_num

while count < total_suf_sa_num:

  bulk = generate_test_bulk(all_pbx_stable_entries)
  SA = generate_random_sa_metal(all_metal_name_list)
  slab_start = time.time()
  random_slab=get_random_clean_slabs(bulk, 1, 1, 6, 15, lll_reduce=False, in_unit_planes=False, primitive=False,
                            max_normal_search=None, min_lw=8.0, xsite='O', height=0.9,
                            r=None, max_ssize=80, check_polar=True) # bulk, n_slabs, max_index, min_slab_size
  if not random_slab:
      print('NO VIABLE CLEAN SLABS')
      continue
  slab_end = time.time()
  print('Sampled %s clean slabs in %.3f' % (random_slab[0].formula, slab_end-slab_start))
  ads_start = time.time()
  # now get random adsorbed slabs
  rand_slab= random.choice(random_slab)
  mxide_adsgen = MXideAdsorbateGenerator(random_slab[0], height=1.25, tol=2, ads_dist_is_blength=False, ads_dist=4,)
  adslab=mxide_adsgen.generate_random_adsorption_structure(SA, 1, max_coverage=1)
  if not adslab:
      print('NO VIABLE adslab')
      continue
  #################### ASE format and fix atom ############
  ase_adslab=AseAtomsAdaptor.get_atoms(adslab[0])
  z_positions=[atom.position[2] for atom in ase_adslab]
  c = FixAtoms(indices=[atom.index for atom in ase_adslab if atom.position[2] <np.median(z_positions)])
  ase_adslab.set_constraint(c)
  #########################################################
  ase_slab=AseAtomsAdaptor.get_atoms(random_slab[0])
  ase_slab_z_positions=[atom.position[2] for atom in ase_slab]
  c = FixAtoms(indices=[atom.index for atom in ase_slab if atom.position[2] <np.median(ase_slab_z_positions)])
  ase_slab.set_constraint(c)
  #########################################################

  #visual(ase_adslab, './', (300, 300),2)
  subdirectory = f'{count}'
  out_directory_adslab
  adslab_subdir_path = os.path.join(out_directory_adslab, subdirectory)
  slab_subdir_path = os.path.join(out_directory_slab, subdirectory)
  os.makedirs(adslab_subdir_path, exist_ok=True)
  os.makedirs(slab_subdir_path, exist_ok=True)
  vasp.write_vasp_input_files(ase_adslab, outdir=adslab_subdir_path, vasp_flags=None)
  vasp.write_vasp_input_files(ase_slab, outdir=slab_subdir_path, vasp_flags=None)

  ads_end=time.time()

  print(f'saved{count} out of 300 in {slab_end-slab_start}')
  slab_list.append(ase_slab)
  adslab_list.append(ase_adslab)

  count= count + 1


### 이제 다운로드 받고, gemnet으로 돌려보기

In [ ]:
!zip -r output_300.zip output_500

In [ ]:
from ase.io import Trajectory, write
write('./output_ase/adslab_list.traj',adslab_list)
write('./output_ase/slab_list.traj',slab_list)

In [ ]:
!zip -r output_ase_before_relax.zip output_ase

  adding: output_ase/ (stored 0%)
  adding: output_ase/slab_list.traj (deflated 75%)
  adding: output_ase/adslab_list.traj (deflated 75%)


In [ ]:
gemnet = get_checkpoint('GemNet-OC OC20+OC22')
from ocpmodels.common.relaxation.ase_utils import OCPCalculator
calc = OCPCalculator(checkpoint_path=os.path.expanduser(gemnet), cpu=True)

/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [ ]:
!cp -r output_300.zip /content/drive/MyDrive/sa_generation_1029

In [ ]:
!cp -r output_ase /content/drive/MyDrive/sa_generation_1029

In [ ]:
from ase.optimize import LBFGS
from ase.build import fcc111, molecule, add_adsorbate
from ase.constraints import ExpCellFilter, StrainFilter

def energy_cal(atoms, fmax=0.05):
    atoms.set_calculator(calc)
    tags = np.ones(len(atoms))
    atoms.set_tags(tags)
    print(tags)
    opt1 = LBFGS(atoms)
    opt1.run(fmax=fmax)
    return atoms.get_total_energy()
def status(proceed):
    num = int(proceed/5)
    return '🟩'*num+'⬜️'*(20-num)
def oc_calculator(data, fmax_ref, calculator):
    E_tot_list=[]
    for i, d in enumerate(data):
        print('<<<<<<<<<<<<<<<<<<<<<<<<<<<<<', i+1, ' out of ', len(data), '>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
        E_tot = energy_cal(d, fmax=fmax_ref)
        E_tot_list.append(round(E_tot, 3))
        proceed = round((i+1) / len(data) * 100, 2)
        print(status(proceed), proceed, '% done')
    return E_tot_list

e_adslab_tot=oc_calculator(adslab_list,0.05,calc)
e_slab_tot=oc_calculator(slab_list,0.05,calc)

<<<<<<<<<<<<<<<<<<<<<<<<<<<<< 1  out of  300 >>>>>>>>>>>>>>>>>>>>>>>>>>>>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.]


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


       Step     Time          Energy         fmax
LBFGS:    0 10:54:20     -332.383453        2.9147
LBFGS:    1 10:54:22     -332.986115        2.3258
LBFGS:    2 10:54:23     -333.911957        5.1333
LBFGS:    3 10:54:24     -334.466675        0.9354
LBFGS:    4 10:54:26     -334.632965        0.6657
LBFGS:    5 10:54:28     -334.735901        0.8263
LBFGS:    6 10:54:30     -334.732300        0.5414
LBFGS:    7 10:54:32     -334.798004        0.5122
LBFGS:    8 10:54:33     -334.850067        0.3800
LBFGS:    9 10:54:35     -334.923401        0.3248
LBFGS:   10 10:54:36     -334.971649        0.3983
LBFGS:   11 10:54:38     -335.069000        0.3648
LBFGS:   12 10:54:39     -335.118927        0.2740
LBFGS:   13 10:54:40     -335.172363        0.2795
LBFGS:   14 10:54:42     -335.235168        0.3010
LBFGS:   15 10:54:44     -335.284302        0.3595
LBFGS:   16 10:54:45     -335.307678        0.3503
LBFGS:   17 10:54:47     -335.287537        0.2767
LBFGS:   18 10:54:49     -335.30

KeyboardInterrupt: ignored



```
from ocdata.oc22_dataset.surface import Slab, SlabGenerator, generate_all_slabs
from pymatgen.entries.computed_entries import ComputedStructureEntry

import time
import random

json_file_path = "unary_list.json"
all_pbx_stable_entries = [ComputedStructureEntry.from_dict(d) for d in \
                          json.load(open(json_file_path))]


def generate_test_bulk(all_pbx_stable_entries, seed=0):
    random.seed(seed)
    tot_start = time.time()
    entry = random.sample(all_pbx_stable_entries, 1)[0]
    test_bulk = entry.structure
    print('Bulk:',test_bulk.formula)  # You can remove this line if you don't want to print the result
    return test_bulk
def generate_random_sa_metal(all_metal_name_list, seed=0):
    random.seed(seed)
    tot_start = time.time()
    entry = random.sample(all_metal_name_list, 1)[0]
    sa = Molecule([entry], [[0, 0, 0]])
    print('SA:',sa.formula)  # You can remove this line if you don't want to print the result
    return sa

from ocdata.oc22_dataset.MXide_adsorption import MXideAdsorbateGenerator
from ocdata.oc22_dataset.termination_generator import get_random_clean_slabs
from pymatgen.core.structure import Structure, Lattice, Molecule
from pymatgen.io.ase import AseAtomsAdaptor
from ase.constraints import FixAtoms

bulk=generate_test_bulk(all_pbx_stable_entries)
SA = generate_random_sa_metal(all_metal_name_list)

random_slab=get_random_clean_slabs(bulk, 1, 2, 10, 15, lll_reduce=False, in_unit_planes=False, primitive=False,
                           max_normal_search=None, min_lw=8.0, xsite='O', height=0.9,
                           r=None, max_ssize=100, check_polar=True) # bulk, n_slabs, max_index, min_slab_size,
print()
mxide_adsgen = MXideAdsorbateGenerator(random_slab[0], height=1.25, repeat=[1,1,1], r=random)
adslab=mxide_adsgen.generate_random_adsorption_structure(SA, 1, max_coverage=1)

ase_adslab=AseAtomsAdaptor.get_atoms(adslab[0])
z_positions=[atom.position[2] for atom in ase_adslab]
c = FixAtoms(indices=[atom.index for atom in ase_adslab if atom.position[2] <np.median(z_positions)])
ase_adslab.set_constraint(c)
visual(ase_adslab, './', (300, 300),2)

from ocdata import vasp
import os
out_directory = './output_dir'
subdirectory = '1'
subdir_path = os.path.join(out_directory, subdirectory)
os.makedirs(subdir_path, exist_ok=True)
vasp.write_vasp_input_files(ase_adslab, outdir=subdir_path, vasp_flags=None)

```




이렇게도 만들 수 있음.
```
slab_list=generate_all_slabs(test_bulk,1,10,15) #structure, max_index, min_slab_size, min_vacuum_size,bonds=None, tol=0.1, ftol=0.1
print('===========================================')
print(len(slab_list))
indices=slab_list[0].get_surface_sites()

print(indices['top'])
random_site = random.choice(indices['top'])
# Print the random site
print(random_site)
last_element = random_site[-1]
print(last_element)

slab_list[0].add_adsorbate_atom([last_element],'Pt',0.9)
slab_list[0]
print(len(indices['top']))
print(len(slab_list))
a=slab_list[0].num_sites  # 이걸로 if statement만들기 cutoff max num atom < 100
print(a)
```

